In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [2]:
DATA_FOLDER = '../data/'
DATA_ZIP = DATA_FOLDER + 'datasets.zip'

DATA_TRAIN_PATH = DATA_FOLDER + 'train.csv'
DATA_TRAIN_PATH_CLEAN = DATA_FOLDER + 'train_clean.csv'
DATA_TEST_PATH = DATA_FOLDER + 'test.csv' 

## Load the training data into feature matrix, class labels, and event ids:

In [3]:
from proj1_helpers import *
from split_data import *
from data_processing import *

y, tX, ids = load_csv_data(DATA_TRAIN_PATH)
y_clean, tX_clean, ids_clean = clean_training(y, tX, ids)


split_ratio = 0.8
tX_train, tX_validation, y_train, y_validation = split_data(tX, y, split_ratio)
tX_train_clean, tX_validation_clean, y_train_clean, y_validation_clean = split_data(tX_clean, y_clean, split_ratio)

TODO: only remove outliers when max - min > threshold like 10
    Doubt with DER_deltar_tau_lep and PRI_jet_all_pt


In [4]:
# from proj1_helpers import *
# from split_data import *

# y, tX, ids = load_csv_data(DATA_TRAIN_PATH)
# y_clean, tX_clean, ids_clean = load_csv_data(DATA_TRAIN_PATH_CLEAN)

# split_ratio = 0.8
# tX_train, tX_validation, y_train, y_validation = split_data(tX, y, split_ratio)
# tX_train_clean, tX_validation_clean, y_train_clean, y_validation_clean = split_data(tX_clean, y_clean, split_ratio)

In [5]:
print("Raw set : ")
row_format = "{:<15}" * 3
print(row_format.format("", "Training", "Validation"))
print(row_format.format("Features", str(tX_train.shape), str(tX_validation.shape)))
print(row_format.format("Labels", str(y_train.shape), str(y_validation.shape)))

print("\nClean set : ")
row_format = "{:<15}" * 3
print(row_format.format("", "Training", "Validation"))
print(row_format.format("Features", str(tX_train_clean.shape), str(tX_validation_clean.shape)))
print(row_format.format("Labels", str(y_train_clean.shape), str(y_validation_clean.shape)))

Raw set : 
               Training       Validation     
Features       (200000, 30)   (50000, 30)    
Labels         (200000,)      (50000,)       

Clean set : 
               Training       Validation     
Features       (35536, 29)    (8884, 29)     
Labels         (35536,)       (8884,)        


## Do your thing crazy machine learning thing here :) ...

### Gradient Descent

In [6]:
from stochastic_gradient_descent import *
from gradient_descent import *
from costs import *


max_iters = 100
gamma = 0.01
initial_w = np.zeros(tX_train_clean.shape[1])

loss, weights_gd = gradient_descent(y_train_clean, tX_train_clean, initial_w, max_iters, gamma)
print(loss)
loss, weights_sgd = stochastic_gradient_descent(y_train_clean, tX_train_clean, initial_w, max_iters, gamma)
print(loss)

0.34483741738725987
0.34177970796002916


In [7]:
# Not compared with the not cleaned set, because it needs normalizing to work
print("GD Training loss : {}".format(compute_loss(y_train_clean, tX_train_clean, weights_gd)))
print("GD Validation loss : {}".format(compute_loss(y_validation_clean, tX_validation_clean, weights_gd)))
print("SGD Training loss : {}".format(compute_loss(y_train_clean, tX_train_clean, weights_sgd)))
print("SGD Validation loss : {}".format(compute_loss(y_validation_clean, tX_validation_clean, weights_sgd)))

GD Training loss : 0.3447192031684568
GD Validation loss : 0.34342161403433336
SGD Training loss : 0.345007073088848
SGD Validation loss : 0.3434787185463747


### Least Squares

In [8]:
from least_squares import *

weights, t_loss = least_squares(y_train, tX_train)
weights_clean, t_loss_clean = least_squares(y_train_clean, tX_train_clean)

In [9]:
print("Training loss : {}".format(compute_loss(y_train, tX_train, weights)))
print("Validation loss : {}".format(compute_loss(y_validation, tX_validation, weights)))
print("Clean training loss : {}".format(compute_loss(y_train_clean, tX_train_clean, weights_clean)))
print("Clean validation loss : {}".format(compute_loss(y_validation_clean, tX_validation_clean, weights_clean)))

Training loss : 0.3394268495585387
Validation loss : 0.3407678460856664
Clean training loss : 0.3357623766686237
Clean validation loss : 0.33477092497039646


### Ridge Regression

In [10]:
from ridge_regression import *
lambda_ = 1

weights, loss = ridge_regression(y_train, tX_train, lambda_)
weights_clean, loss_clean = ridge_regression(y_train_clean, tX_train_clean, lambda_)

In [11]:
print("Training loss : {}".format(compute_loss(y_train, tX_train, weights)))
print("Validation loss : {}".format(compute_loss(y_validation, tX_validation, weights)))
print("Clean training loss : {}".format(compute_loss(y_train_clean, tX_train_clean, weights_clean)))
print("Clean validation loss : {}".format(compute_loss(y_validation_clean, tX_validation_clean, weights_clean)))

Training loss : 0.3394307923481362
Validation loss : 0.3407529146909622
Clean training loss : 0.34000161105674515
Clean validation loss : 0.3388098362908591


### Logistic Regression

In [12]:
from logistic_regression import *

max_iters = 50
gamma = 0.1
initial_w_train = np.ones(tX_train.shape[1])
initial_w_clean = np.ones(tX_clean.shape[1])

weights, loss = logistic_regression(y_train, tX_train, initial_w_train, max_iters, gamma) 
weights_clean, loss_clean = logistic_regression(y_clean, tX_clean, initial_w_clean, max_iters, gamma) 

In [13]:
print("Training loss : {}".format(calculate_loss(y_train, tX_train, weights)))
print("Validation loss : {}".format(calculate_loss(y_validation, tX_validation, weights)))
print("Clean training loss : {}".format(calculate_loss(y_train_clean, tX_train_clean, weights_clean)))
print("Clean validation loss : {}".format(calculate_loss(y_validation_clean, tX_validation_clean, weights_clean)))

Training loss : inf
Validation loss : inf
Clean training loss : inf
Clean validation loss : inf


### Regularized Logistic Regression

## Generate predictions and save ouput in csv format for submission:

In [15]:
#This enable to run all cells without running this one when unnecessary
if True :
    y_test, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)
    y_test_clean, tX_test_clean, ids_test_clean = clean_test(y_test, tX_test, ids_test)
    OUTPUT_PATH = DATA_FOLDER + 'submission.csv' 
    y_pred = predict_labels(weights_clean, tX_test_clean)
    create_csv_submission(ids_test_clean, y_pred, OUTPUT_PATH)
else :
    print("Change False to True to generate prediction")

In [16]:
print(len(y_test), len(y_pred))

568238 568238
